# Using Swarm Ochestration

You can set the speaker selection method to `swarm` to use the swarm-like orchestration.

- **Headoffs**: Agents can transfer control to another agent via function calls, enabling smooth transitions within workflows.
- **Context Variables**: Agents can dynamically update shared variables through function calls, maintaining context and adaptability throughout the process.




## SwarmAgent and SwarmResult

To allow easier setup of the swarm, a `SwarmAgent` class is provided. When passing functions to this agent, the agent will execute any tool calls from LLMs directly. 

```python
agent = SwarmAgent(
    ..., # same arguments as AssistantAgent
    functions=[func_1, func_2, ...] # a new parameter that allows passing a list of functions
)
```

We also create a `SwarmResult` class, where you can return a value, the next agent to call and update context variables at the same time.

```python
def func_name() -> SwarmResult:
    return SwarmResult(value="value", agent=<swarmagent object>, context_variables={"key": "value"})
```

To only define transitions between agents, we provide an quick way to define the handoffs:

```python
swarm_agent.handoff(agent, condition="when should transition happen")
```
We will register a transfer function to llm config:
```
def transfer_to_agent_name():
    """<Condition string>"""
    return agent_name
```

When defining function calls, you can return one of the following types:
- `str`
- `SwarmAgent`: if a `SwarmAgent` object is returned, it will be called next.
- `SwarmResult`: if a `SwarmResult` object is returned, we will use it to update the context variables, return a value and call the next agent.


**Notes for creating the function calls**
- For input arguments, you must define the type of the argument, otherwise the registration will fail (e.g. `arg_name: str`). 
- The docstring of the function will be used as the prompt. So make sure to write a clear description.
- The function name will be used as the tool name.


In [ ]:
from autogen.agentchat.contrib.swarm_agent import SwarmAgent, SwarmResult

llm_config = "Setup your config here"


def update_context_1(context_variables: dict) -> str:
    context_variables["1"] = True
    return SwarmResult(
        value="success", context_variables=context_variables
    )  # Update context variables and return success


def update_context_2_and_transfer_to_3(context_variables: dict) -> str:
    context_variables["2"] = True
    return SwarmResult(
        value="success", context_variables=context_variables, agent=agent_3
    )  # Update context variables, return success, and transfer to agent_3


def update_context_3(context_variables: dict) -> str:
    context_variables["3"] = True
    return SwarmResult(
        value="success", context_variables=context_variables
    )  # Update context variables and return success


agent_1 = SwarmAgent(
    name="Agent_1",
    system_message="You are Agent 1, first, call the function to update context 1, and transfer to Agent 2",
    llm_config=llm_config,
    functions=[update_context_1],
)

agent_2 = SwarmAgent(
    name="Agent_2",
    system_message="You are Agent 2, call the function that updates context 2 and transfer to Agent 3",
    llm_config=llm_config,
    functions=[update_context_2_and_transfer_to_3],
)

agent_3 = SwarmAgent(
    name="Agent_3",
    system_message="You are Agent 3, first, call the function to update context 3, and then reply TERMINATE",
    llm_config=llm_config,
    functions=[update_context_3],
)

agent_1.hand_off(agent_2, condition="handoff to agent 2")  # register handoff using hand_off method.

## Use initialize_swarm_chat

To get response from the swarm agent, you can use the `initialize_swarm_chat` function. This function will return the response from the agent and the updated context variables.

```python
chat_history, context_variables, last_agent = initialize_swarm_chat(
    init_agent, # which agent to start with
    messages, # list of messages
    agents, # pass in all the swarm agents
    max_rounds, # maximum number of rounds
    context_variables, # initial context variables
)
```


In [ ]:
from autogen.agentchat.contrib.swarm_agent import initialize_swarm_chat

context_variables = {"1": False, "2": False, "3": False}
chat_result, context_variables, last_agent = initialize_swarm_chat(
    init_agent=agent_1,
    agents=[agent_1, agent_2, agent_3],
    messages="start",
    context_variables=context_variables,
)
print(context_variables)